In [1]:
!module load cuda

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from jupyterplot import ProgressPlot
from tqdm import tqdm, trange

from datetime import datetime, timedelta
import seaborn as sns

In [3]:
class WeatherDataset(Dataset):
    def __init__(self, weather_data = 'data/big_arpafvg.csv', img_dir = 'data/images'):
        initial_data=pd.read_csv(weather_data)
        initial_data = initial_data.drop(columns=['Temp. min gradi C','Temp. med gradi C','Temp. max gradi C','Vento med km/h','Dir. V. max gradi N'])
        #normalize the data
        for col in initial_data.columns:
            if col != 'giorno' and col != 'mese' and col != 'anno':
                initial_data[col] = (initial_data[col] - initial_data[col].mean()) / initial_data[col].std()
        self.weather_data = initial_data
        self.img_dir = img_dir
        self.seq_length = 7
        self.target_column_index = 3
        self.date_generated = []

    def __getitem__(self, date):
        #get day month year from date in format dd-mm-yyyy
        day, month, year = date.split('_')
        #get from weather data the row with the same date
        weather_data = self.weather_data[(self.weather_data['giorno'] == int(day)) & (self.weather_data['mese'] == int(month)) & (self.weather_data['anno'] == int(year))]
        weather_data = torch.tensor(weather_data.values[0])

        image = os.path.join(self.img_dir, str(date) + '.jpg')  
        image = read_image(image)   
        return [image, weather_data]
    
    def __len__(self):
        return len(self.weather_data)

    def date_generation(self, start_date, end_date):
        start_date = datetime.strptime(start_date, "%d_%m_%Y")
        end_date = datetime.strptime(end_date, "%d_%m_%Y")
        date_generated = [start_date + timedelta(days=x) for x in range(0, (end_date-start_date).days)]
        #transform the list of datetime objects in a list of strings in format dd-mm-yyyy
        date_generated = [date.strftime("%d_%m_%Y") for date in date_generated]
        self.date_generated=date_generated
        return date_generated
    
    def create_sequence(self, date, len_seq):
        batch_xs =[]
        batch_imgs = []
        batch_ys = []
        for elem in date:
            ys = []
            end_of_week = elem.split('_')
            end_of_week = datetime(int(end_of_week[2]), int(end_of_week[1]), int(end_of_week[0])) + timedelta(days = len_seq)
            end_of_week = end_of_week.strftime("%d_%m_%Y")
            imgs, xs =[], []
            week=self.date_generation(elem,end_of_week)
            for day in week:
                a , b = self.__getitem__(day)
                imgs.append(a)
                xs.append(b)


            next = end_of_week.split('_')
            next = datetime(int(next[2]), int(next[1]), int(next[0])) + timedelta(days = 1)
            next = next.strftime("%d_%m_%Y")
            ys.append(self.__getitem__(next)[1][self.target_column_index])
            imgs = np.array(imgs)
            xs = np.array(xs)
            ys = np.array(ys)
            imgs = torch.from_numpy(imgs).float()
            xs = torch.from_numpy(xs).float()
            ys = torch.from_numpy(ys).float()
            batch_xs.append(xs)
            batch_imgs.append(imgs)
            batch_ys.append(ys)
                    
        batch_xs = np.array(batch_xs)
        batch_imgs = np.array(batch_imgs)
        batch_ys = np.array(batch_ys)
        batch_xs = torch.from_numpy(batch_xs).float()
        batch_imgs = torch.from_numpy(batch_imgs).float()
        batch_ys = torch.from_numpy(batch_ys).float()
        return batch_imgs, batch_xs, batch_ys

In [4]:
dataset = WeatherDataset()

In [22]:
#generate a list of strings dd-mm-yyyy from 01-06-2023 to 15-6-2024
dataset.date_generation("01_01_2023", "08_06_2024")
date_generated = dataset.date_generated


In [25]:
train_len = int(0.8 * len(dataset.date_generated))+1
test_len = len(dataset.date_generated) - train_len
#split the dataset: date_trainset contains the first 80% of the dates, date_testset contains the remaining 20%
date_trainset = date_generated[:train_len]
date_testset = date_generated[train_len:]
print(train_len)
print(test_len)

420
104


In [17]:
train_loader = torch.utils.data.DataLoader(date_trainset, batch_size = 4, shuffle = True)
test_loader = torch.utils.data.DataLoader(date_testset, batch_size = 4, shuffle = True)

In [18]:
class DeepWeather(nn.Module):
    def __init__(self):
        super(DeepWeather, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 21, out_channels = 32, kernel_size = (3, 3), stride = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(p = 0.3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (3, 3), stride = 1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = (3, 3), stride = 1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = (3, 3), stride = 1)
        self.bn6 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(6154, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 1)

        self.input_size = 10
        self.hidden_size = 16
        self.num_layers = 1
        self.output_size = 16
        self.rnn = nn.RNN(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.fc = nn.Linear(230416, 1)

    def forward(self, x1, x2):
        print(x1.shape)
        # Flatten the last two dimensions and add them together
        x1 = x1.view(5,7*3,512,512)
        
        x1 = self.bn1(self.dropout(self.pool(F.leaky_relu(self.conv1(x1)))))
        x1 = self.bn4(self.dropout(self.pool(F.leaky_relu(self.conv4(x1)))))
        x1 = self.bn5(self.dropout(self.pool(F.leaky_relu(self.conv5(x1)))))
        x1 = self.bn6(self.dropout(self.pool(F.leaky_relu(self.conv6(x1)))))
        x1 = torch.flatten(x1, start_dim = 1)

        h0 = torch.zeros(self.num_layers, x2.size(0),self.hidden_size).to(x2.device)
        out, _ = self.rnn(x2, h0)
        out = torch.cat((out[:,-1,:], x1), dim = 1)
        out = self.fc(out)

        return out

In [13]:
def train(model, dataset, train_loader, test_loader, criterion, optimizer, epochs):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = model.to(device)    
    
    train_loss_epochs = []
    test_loss_epochs = []
    bar = trange(epochs, desc=f"Epoch ?/?, Train Loss: ?, Test Loss: ?")
    for epoch in bar:
        model.train()
        train_losses = []
            
        for date in train_loader:
            imgs, xs, ys = dataset.create_sequence(date, 7)
            imgs = imgs.to(device)
            xs = xs.to(device)
            ys = ys.to(device)

            optimizer.zero_grad()
            outputs = model(imgs, xs)
            loss = criterion(outputs, ys)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            
        train_loss_epochs.append(np.mean(train_losses))
        model.eval()
        test_losses = []
        for date in train_loader:
            
            imgs, xs, ys = dataset.create_sequence(date, 7)
            imgs = imgs.to(device)
            xs = xs.to(device)
            ys = ys.to(device)
            outputs = model(imgs, xs)
            loss = criterion(outputs.squeeze(), ys)
            test_losses.append(loss.item())
                
        test_loss_epochs.append(np.mean(test_losses))
        bar.set_description(f"Epoch {epoch + 1}/{epochs}, Train Loss: {np.mean(train_losses)}, Test Loss: {np.mean(test_losses)}")
        #if first_time:
            #torch.save(model.state_dict(), f"deepweather_epoch{num_saved_epochs + epoch + 1}.pth")

    return train_loss_epochs, test_loss_epochs

In [19]:
model = DeepWeather()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 10e-6, weight_decay=1e-5)

In [20]:
train_losses, val_losses = train(model, dataset, train_loader, test_loader, criterion, optimizer, epochs = 50)
#torch.save(model.state_dict(), "models/model_2dcnn_rnn.tensor")

Epoch ?/?, Train Loss: ?, Test Loss: ?:   0%|                                                    | 0/50 [00:00<?, ?it/s]

torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size([10, 7, 3, 512, 512])
torch.Size

Epoch ?/?, Train Loss: ?, Test Loss: ?:   0%|                                                    | 0/50 [00:17<?, ?it/s]

torch.Size([9, 7, 3, 512, 512])


RuntimeError: shape '[10, 21, 512, 512]' is invalid for input of size 49545216

In [ ]:
epochs = [epoch for epoch in range(50)]
plt.plot(epochs, train_losses, label='Training Loss', color='blue')
plt.plot(epochs, val_losses, label='Validation Loss', color='mediumaquamarine')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title("2D CNN RNN")
plt.savefig("plots/2dcnn_rnn_loss_batch5.png")
plt.show()


In [ ]:
#state_dict=torch.load("models/model_2dcnn_rnn.tensor")
#model.load_state_dict(state_dict)

In [ ]:
prediction_dataset = WeatherDataset()
prediction_dataset.date_generation('24_12_2023','08_06_2024')
week=prediction_dataset.date_generated
pred_loader=torch.utils.data.DataLoader(week, batch_size = 1, shuffle = False)

In [ ]:
def predict(model, pred_loader, dataset):
    predictions=[]
    predicted_days=[]
    real_rains=[]
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model=model.to(device)
    for date in pred_loader:  
            imgs, xs, ys = dataset.create_sequence(date, 7)
            imgs = imgs.to(device)
            xs = xs.to(device)
            ys = ys.to(device)
            output = model(imgs, xs)
            output=output.cpu()
            predictions.append(output.detach().numpy())
            predicted_day = date[0].split('_')
            predicted_day = datetime(int(predicted_day[2]), int(predicted_day[1]), int(predicted_day[0])) + timedelta(days = 8)
            predicted_day = predicted_day.strftime("%d_%m_%Y")
            predicted_days.append(predicted_day)
            real_rain = dataset[predicted_day][1][3]
            real_rains.append(real_rain)
            
    return predicted_days, predictions, real_rains

In [ ]:
predicted_days, predictions, real_rain= predict(model, pred_loader, prediction_dataset)

In [ ]:
predicted_days = np.array(predicted_days)
real_rain = np.array(real_rain)

date_to_display = ['01_01_2024', '01_02_2024', '01_03_2024', '01_04_2024', '01_05_2024', '01_06_2024']

indices_to_display = [np.where(predicted_days == date)[0][0] for date in date_to_display if date in predicted_days]

plt.figure(figsize=(10, 6))
sns.lineplot(x=predicted_days, y=real_rain, label='Real rain', color='dodgerblue')
sns.lineplot(x=predicted_days, y=np.array(predictions).squeeze(), label='Predictions', color='navy')

plt.xticks(predicted_days[indices_to_display], date_to_display, fontsize=8)

plt.title('Predictions 2D CNN-RNN')
plt.legend()
plt.savefig("plots/2dcnn_rnn_pred_batch5.png")
plt.show()
